In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import trim, col

In [0]:
df = spark.table("workspace.bronze.erp_loc_a101")

In [0]:
#Trimming
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
#CID cleanups
df = df.withColumn("cid", F.regexp_replace(col("cid"), "-", ""))

In [0]:
#Country normalization
df = df.withColumn(
    "cntry",
    F.when(col("cntry") == "DE", "Germany")
     .when(col("cntry").isin("US", "USA"), "United States")
     .when((col("cntry") == "") | col("cntry").isNull(), "n/a")
     .otherwise(col("cntry"))
)
     

In [0]:
#Renaming columns
RENAME_MAP = {
    "cid": "customer_number",
    "cntry": "country"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)
     


In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")